In [8]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ---------------------------------------
# 1. Create synthetic dataset
# ---------------------------------------
np.random.seed(0)
n = 200
experience = np.random.randint(1, 30, size=n)
education_level = np.random.choice(['Bachelors', 'Masters', 'PhD'], size=n)
salary = 25000 + (experience ** 1.5) * 300 + np.random.normal(0, 10000, size=n)
salary[::10] = np.nan  # introduce missing values

data = pd.DataFrame({
    'Experience': experience,
    'Education': education_level,
    'Salary': salary
})

In [9]:
# Display first few rows
data.head()

Experience  Education        Salary
0          13  Bachelors           NaN
1          16    Masters  49076.594292
2          22        PhD  52198.467065
3           1  Bachelors  27578.507663
4           4  Bachelors  34437.886819

In [10]:
# 2. Handle outliers (Cap method)
# ---------------------------------------
def cap_outliers(df):
    df = df.copy()
    q_low = df['Salary'].quantile(0.01)
    q_hi = df['Salary'].quantile(0.99)
    df['Salary'] = np.clip(df['Salary'], q_low, q_hi)
    return df

data = cap_outliers(data)

In [11]:
# 3. Split into features and target
# ---------------------------------------
X = data.drop('Salary', axis=1)
y = data['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# 4. Build Preprocessing Pipeline
# ---------------------------------------
numeric_features = ['Experience']
categorical_features = ['Education']

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


In [13]:
# 5. Train SVR Model with RBF kernel
# ---------------------------------------
svr_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('svr', SVR(kernel='rbf', C=100, epsilon=5000))
])

# Impute missing values in y_train and y_test
imputer = SimpleImputer(strategy='mean')
y_train_imputed = imputer.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_imputed = imputer.transform(y_test.values.reshape(-1, 1)).ravel()


svr_pipeline.fit(X_train, y_train_imputed)
y_pred = svr_pipeline.predict(X_test)

# Evaluation
print("\n--- SVR Model Evaluation ---")
print("R² Score:", r2_score(y_test_imputed, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test_imputed, y_pred)))



--- SVR Model Evaluation ---
R² Score: 0.15583706411284837
RMSE: 13603.83598839538


In [14]:
# 7. Predict on New Data
# ---------------------------------------
new_data = pd.DataFrame({
    'Experience': [10, 20],
    'Education': ['Masters', 'PhD']
})

predicted_salaries = svr_pipeline.predict(new_data)
print("\n--- Predicted Salaries for New Data ---")
print(predicted_salaries)


--- Predicted Salaries for New Data ---
[40795.16831063 42988.43374453]
